# CIP Assignment 2

## Garima Jain
## M.Tech AI
## 18MCMI14

In [330]:
import cv2 as cv
import numpy as np
import glob

### Q1. Search the web for Colour Halftoning algorithms. Select one of them and write a detailed report on it OR implement the selected algorithm and show results on the test images.

Implementing CMY Ordered Dot Dithering Algorithm

In [331]:
'''
output_size = 16 x input_size 
4x4 CMY color dithered mask 
set threshold, which pixels will be set and which not
for each pixel in input image, calculate #dots to be put for C,M,Y acc. to corr. pixel vlaues
'''

def cmy_color_dithering(image,result_image_name):
    output_img = np.zeros((image.shape[0]*4, image.shape[1]*4, 3), np.uint8)

    #indexes for each in priority order according to Bayer's mask
    ordered_c_ind = [[2,2],[0,2],[2,0],[3,1],[2,3]] 
    ordered_m_ind = [[1,1],[1,3],[0,1],[0,3],[2,1],[1,2]]
    ordered_y_ind = [[0,0],[3,3],[1,0],[3,2],[3,0]]
    
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            c_val = 255 - image[i,j,2]
            m_val = 255 - image[i,j,1]
            y_val = 255 - image[i,j,0]
            
            #find number of dots to be set for C,M,Y using unitary method 
            c_dots = ( len(ordered_c_ind) * c_val ) // 255
            m_dots = ( len(ordered_m_ind) * m_val ) // 255
            y_dots = ( len(ordered_y_ind) * y_val ) // 255
            
            x,y = i*4, j*4
            for k in range(c_dots): # for cyan, set B and G 
                output_img[x+ordered_c_ind[k][0],y+ordered_c_ind[k][1]] = [255,255,0]

            for k in range(m_dots): # for magenta, set B and R
                output_img[x+ordered_m_ind[k][0],y+ordered_m_ind[k][1]] = [255,0,255]
            
            for k in range(y_dots): # for yellow, set G and R
                output_img[x+ordered_y_ind[k][0],y+ordered_y_ind[k][1]] = [0,255,255]
    
    cv.imwrite(result_image_name,output_img)

In [332]:
'''
mask:
[Y1,M3,C2,M4]
[Y3,M1,M6,M2]
[C3,M5,C1,C5]
[Y5,C4,Y4,Y2]

C = W-R = G+B
M = W-G = R+B
Y = W-B = R+G
'''
image = cv.imread('Test_data/waterplane.ppm')
cmy_color_dithering(image,'Q1.CMYDithering/waterplane.ppm')

image = cv.imread('Test_data/orange-flower.ppm')
cmy_color_dithering(image,'Q1.CMYDithering/orange-flower.png')

image = cv.imread('Test_data/fall-colours.jpg')
cmy_color_dithering(image,'Q1.CMYDithering/fall-colours.jpg')

The output images are in folder Q1.CMYDithering.
The mask I used is shown above.

### Q2. Try coming up with your own error diffusion coefficients and implement the standard error-diffusion algorithm. Compare the performance of your coefficients against Floyd-Steinberg's on this image. Discuss the patterns visible in yours and in Floyd-Steinberg's at the various gray levels.

In [333]:
def error_diffusion_algo(image, indexes, num_of_indexes, coeffs, result_image):
    new_image = image.copy()
    for i in range(1,new_image.shape[0]-2):
        for j in range(1,new_image.shape[1]-2):
            error = 0
            if new_image[i][j] < 128 : # threshold for binarisation - 128
                error = new_image[i][j]
                new_image[i][j] = 0                
            else:
                error = new_image[i][j] - 255
                new_image[i][j] = 255
            for k in range(num_of_indexes):
                new_image[i+indexes[k][0]][j+indexes[k][1]] = np.clip(
                                                                (new_image[i+indexes[k][0]][j+indexes[k][1]]\
                                                                + coeffs[k] * error), 0, 255)    
#     new_image = np.uint8(new_image)
    cv.imwrite(result_image, new_image)
    return new_image

In [334]:
image = cv.imread('Test_data/ed-eg.png',0)

floyd_steinberg_indexes = [[0,1], [1,1], [1,0], [1,-1]]
floyd_steinberg_coeffs = [7 / 16, 1 / 16, 5 / 16, 3 / 16]
fd_num_of_indexes = 4
error_diffusion_algo(image, floyd_steinberg_indexes, fd_num_of_indexes, floyd_steinberg_coeffs, "Q2.Results/FloydSteinbergDiffused.png")

my_diffusion_indexes = [[0,1], [1,1], [1,0],[0,2], [2,2], [2,0]]
my_num_of_indexes = 6
my_diffused_image = error_diffusion_algo(image, my_diffusion_indexes, my_num_of_indexes, my_diffusion_coeffs, "Q2.Results/MyDiffused.png")

The output images are in folder Ques2.Results

Error diffusing algorithm is a type of half toning in which error is diffused to other pixels, if pixel value < 127, then error is pixel value,else it's pixel value-255. Error is the value by which pixel value deviates from threshold, which is 127. Error is diffused to neighboring pixels and then if it's value is<127, then pixel is not set, i.e black dot else white dot is put. 
Here, first I have implemented error diffuing algorithm with Floyd Steinberg coefficients and then with my own set of coefficients. I am diffusing error to 2 right pixels, 2 bottom pixels and 2 bottom right diagonal pixels. 

Difference in the patterns visible in Floyd Steinberg algorithm and one with my coefficients:In the upper half of the image, floyd steinberg result has some patterns throughout, there are no visible patterns in mine though, it's quite smooth. However, in the lower half of the image, verticals line can be seen in my output at various gray levels while Floyd Steinberg has small patterns as compared to mine. Since I am diffusing error to 2 pixels, elongated patterns are visible in mine.

### Q3. 
### (a).Implement an algorithm to simulate the grayscale output from a colour filter array. The function prototype is image colour_filter (image, filter).That is, it takes an input colour image and a colour filter as parameters and returns a grayscale image. 
### (b). Implement a demosaicking algorithm with the prototype image demosaic (image, filter).The input image is a grayscale image output by the colour_filter algorithm and the corresponding filter array; the output is a colour image.


a. Apply color filter array on color image to get grayscale image.

In [335]:
def color_to_grayscale(image, color_filter, result_image):
    neighbors = [[0,0], [0,1], [1,0], [1,1]]
    new_image = np.zeros((image.shape[0],image.shape[1]))
    # pass 2x2 filter over the image, for each filter component value, 
    # retain that component value in pixel
    for i in range(0, image.shape[0]-2, 2):
        for j in range(0, image.shape[1]-2, 2):
            for k in range(0, 4):
                new_image[i+neighbors[k][0]][j+neighbors[k][1]] = image[i+k][j+k][color_filter[k]]
    cv.imwrite(result_image, new_image)

In [336]:
color_filter1 = [1, 2, 0, 1] # Bayer's mask which Canon uses
color_filter2 = [2, 1, 1, 0] # Bayer's mask which other manufaturers use

image = cv.imread('Test_data/orange-flower.ppm')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
color_filter_2grayscale(image, color_filter1,'Q3.Grayscale_image_results/filter1_orange_flower.jpg')
color_filter_2grayscale(image, color_filter2,'Q3.Grayscale_image_results/filter2_orange_flower.jpg')

image = cv.imread('Test_data/fall-colours.jpg')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
color_filter_2grayscale(image, color_filter1,'Q3.Grayscale_image_results/filter1_fall_colors.jpg')
color_filter_2grayscale(image, color_filter2,'Q3.Grayscale_image_results/filter2_fall_colors.jpg')

image = cv.imread('Test_data/waterplane.ppm')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
color_filter_2grayscale(image, color_filter1,'Q3.Grayscale_image_results/filter1_waterplane.jpg')
color_filter_2grayscale(image, color_filter2,'Q3.Grayscale_image_results/filter2_waterplane.jpg')

The output images are in folder Q3.Grayscale_image_results.

I have used both the filters, the one which Canon uses and the one which other manufacturers use. There are no visible changes at normal level though.

b. Demosaicking algorithm - apply color filter array on grayscale image - convert to color image

In [337]:
def grayscale_to_color(image, color_filter, result_image):
    '''
    pass the filter over the pixels, set values for each channel corr. to mask 
    '''
    new_image = np.zeros((image.shape[0], image.shape[1], 3))    
    for i in range(0, image.shape[0]-1, 1):
        for j in range(0, image.shape[1]-1, 1):
            r = np.where(color_filter == 0)
            g = np.where(color_filter == 1)
            b = np.where(color_filter == 2)
            
            for x,y in zip(r[0],r[1]):
                new_image[i][j][2] = image[i+x][j+y] # red component
            
            for x,y in zip(g[0],g[1]):
                new_image[i][j][1] += image[i+x][j+y]
            new_image[i][j][1] = new_image[i][j][1] / 2
            
            for x,y in zip(b[0],b[1]):
                new_image[i][j][0] = image[i+x][j+y] # blue
            
            color_filter = np.roll(color_filter, shift=1, axis=1)
        
        color_filter = np.roll(color_filter, shift=1, axis=1)
        color_filter = np.roll(color_filter, shift=1, axis=0)
    cv.imwrite(result_image,new_image)

In [338]:
color_filter1 = np.array([[1,2],[0,1]]) # Bayer's mask which Canon uses
color_filter2 = np.array([[2,1],[1,0]]) 

image1 = cv.imread('Q3.Grayscale_image_results/filter1_fall_colors.jpg',0)
image2 = cv.imread('Q3.Grayscale_image_results/filter2_fall_colors.jpg',0)
grayscale_to_color(image1, color_filter1,'Q3.Demosaicking_results/filter1_fall_colors.png')
grayscale_to_color(image2, color_filter2,'Q3.Demosaicking_results/filter2_fall_colors.png')

image1 = cv.imread('Q3.Grayscale_image_results/filter1_orange_flower.jpg',0)
image2 = cv.imread('Q3.Grayscale_image_results/filter2_orange_flower.jpg',0)
grayscale_to_color(image1, color_filter1,'Q3.Demosaicking_results/filter1_orange_flower.png')
grayscale_to_color(image2, color_filter2,'Q3.Demosaicking_results/filter2_orange_flower.png')

image1 = cv.imread('Q3.Grayscale_image_results/filter1_waterplane.jpg',0)
image2 = cv.imread('Q3.Grayscale_image_results/filter2_waterplane.jpg',0)
grayscale_to_color(image1, color_filter1,'Q3.Demosaicking_results/filter1_waterplane.png')
grayscale_to_color(image2, color_filter2,'Q3.Demosaicking_results/filter2_waterplane.png')

The resulting images are in folder Q3.Demosaicking_results.

Again, I applied the algorithm using both the filters. The images produced using Filter 2 seem to be more saturated than the ones produced using filter1. 